In [32]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# Вычисление динамического интервала
def calculate_dynamic_range(atr, current_price, min_range=0.0035, max_range=0.02):
    atr_percent = atr / current_price
    dynamic_range = np.clip(atr_percent * 2, min_range, max_range) # Здесь от множетеля зависит итоговое количество сигналов
    return dynamic_range # значение от 0.0035 до 0.02

def generate_targets(df, max_hold=10):
    targets = []
    for i in range(len(df)):
        entry_price = df.iloc[i]['close']
        dynamic_range = df.iloc[i]['dynamic_range']
        result = 0
        for j in range(1, max_hold+1):
            if i+j >= len(df):
                break
            high = df.iloc[i+j]['high']
            low = df.iloc[i+j]['low']
            close = df.iloc[i+j]['close']

            upper_breach = (high / entry_price - 1) >= dynamic_range / 2
            lower_breach = (1 - low / entry_price) >= dynamic_range / 2

            if upper_breach and lower_breach:
                # Выбор по Close следующего бара, если Low ниже, а High выше закрытия предыдущего бара
                if close > entry_price:
                    result = 1
                    break
                else:
                    result = -1
                    break
            elif upper_breach:
                result = 1
                break
            elif lower_breach:
                result = -1
                break
        targets.append(result)
    return pd.Series(targets, index=df.index)

def prepare_data(df, max_hold=10):
    # Индикаторы
    atr_indicator = vbt.ATR.run(df['close'], df['high'], df['low'], window=14)
    sma_q = vbt.MA.run(df['close'], window=20)
    sma_s = vbt.MA.run(df['close'], window=200)
    df['atr'] = atr_indicator.atr.values  # Извлечение числового значения ATR
    df['returns'] = df['close'] / df['close'].shift(1)
    rsi_indicator = vbt.RSI.run(df['close'], window=14)
    rsi = rsi_indicator.rsi.values
    df['dynamic_range'] = df.apply(lambda x: calculate_dynamic_range(x['atr'], x['close']), axis=1)
    
    # Генерация целевой переменной
    df['target'] = generate_targets(df, max_hold)
    
    # Фичи
    df['range_pct'] = (df['high'].shift() - df['low'].shift()) / df['close'].shift()
    df['body_pct'] = (df['close'].shift() - df['open'].shift()) / df['open'].shift()
    df['sma_10'] = df['close'].rolling(10).mean()
    df['rsi'] = rsi
    df['rev'] = -np.log(df['open'] / df['close'].shift())
    df['mom'] = np.log(df['close'].shift() / df['open'].shift())
    df['SMA_delta'] = sma_s.ma - sma_q.ma
    # Удаление NaN
    df = df.dropna(subset=['target', 'atr', 'rsi', 'rev', 'mom', 'SMA_delta'])
    
    return df

df = pd.read_csv('AAVE_USDT_15m_candles.csv', sep=';', index_col=0)
df = df.sort_values('timestamp').reset_index(drop=True)
prepared_df = prepare_data(df, max_hold=10)

prepared_df['rsi'] = prepared_df['rsi'].shift()
prepared_df['SMA_delta'] = prepared_df['SMA_delta'].shift()
prepared_df['sma_10'] = prepared_df['sma_10'].shift()
prepared_df = prepared_df.dropna(subset=['rsi', 'SMA_delta', 'sma_10'])
prepared_df

C:\Users\User\AppData\Local\Temp\ipykernel_13092\3716786224.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepared_df['rsi'] = prepared_df['rsi'].shift()
C:\Users\User\AppData\Local\Temp\ipykernel_13092\3716786224.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepared_df['SMA_delta'] = prepared_df['SMA_delta'].shift()
C:\Users\User\AppData\Local\Temp\ipykernel_13092\3716786224.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,open,high,low,close,atr,returns,dynamic_range,target,range_pct,body_pct,sma_10,rsi,rev,mom,SMA_delta
200,53.30,53.50,53.20,53.30,0.234309,1.001880,0.008792,-1,0.005639,-0.005607,53.640,30.769231,-0.001878,-0.005623,-1.20850
201,53.30,53.40,53.30,53.30,0.229734,1.000000,0.008620,-1,0.005629,0.000000,53.590,35.714286,-0.000000,0.000000,-1.19000
202,53.30,53.40,53.20,53.40,0.212436,1.001876,0.007956,-1,0.001876,0.000000,53.550,35.714286,-0.000000,0.000000,-1.17150
203,53.30,53.40,52.90,53.00,0.210778,0.992509,0.007954,1,0.003745,0.001876,53.520,42.857143,0.001874,0.001874,-1.15700
204,52.90,53.10,52.90,53.00,0.209341,1.000000,0.007900,1,0.009434,-0.005629,53.450,29.411765,0.001889,-0.005644,-1.12900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34946,112.43,112.48,111.33,112.09,0.713212,0.997153,0.012726,1,0.005960,0.002944,112.395,49.453552,-0.000178,0.002940,1.04415
34947,112.08,112.95,112.02,112.45,0.834117,1.003212,0.014835,-1,0.010260,-0.003024,112.314,44.783715,0.000089,-0.003029,1.00110
34948,112.40,112.51,112.03,112.15,0.788235,0.997332,0.014057,-1,0.008270,0.003301,112.347,52.736318,0.000445,0.003296,0.92620
34949,112.10,112.33,111.49,111.50,0.753804,0.994204,0.013521,0,0.004280,-0.002224,112.325,53.132832,0.000446,-0.002227,0.85815


In [33]:
prepared_df.to_csv('AAVE_USDT.csv', sep=';')

In [6]:
# Рассчитываем корреляции между индикаторами
correlation_matrix = prepared_df.corr()
correlation_matrix

,open,high,low,close,atr,returns,dynamic_range,target,range_pct,body_pct,sma_10,rsi,rev,mom,SMA_delta
open,1.000000,0.999793,0.999785,0.999684,0.606602,-0.009976,0.443416,-0.008853,0.232768,-0.009194,0.999370,0.046154,0.005693,0.014786,-0.177169
high,0.999793,1.000000,0.999637,0.999812,0.614079,0.002918,0.450120,-0.008653,0.244924,0.003892,0.999167,0.050159,0.006859,0.014957,-0.179543
low,0.999785,0.999637,1.000000,0.999831,0.601440,0.004025,0.438617,-0.009249,0.219407,0.004947,0.999186,0.049668,0.006466,0.015309,-0.175225
close,0.999684,0.999812,0.999831,1.000000,0.607882,0.014428,0.444590,-0.009278,0.231275,0.015438,0.999130,0.051966,0.006941,0.014281,-0.176963
atr,0.606602,0.614079,0.601440,0.607882,1.000000,0.054854,0.851217,0.008380,0.568686,0.055502,0.599793,0.185688,0.003732,0.102945,-0.332661
returns,-0.009976,0.002918,0.004025,0.014428,0.054854,1.000000,0.053855,-0.016200,-0.065306,0.990235,-0.007210,0.245982,-0.085693,-0.030521,0.004262
dynamic_range,0.443416,0.450120,0.438617,0.444590,0.851217,0.053855,1.000000,0.021989,0.524884,0.053563,0.438718,0.195153,-0.002905,0.089726,-0.196383
target,-0.008853,-0.008653,-0.009249,-0.009278,0.008380,-0.016200,0.021989,1.000000,0.021340,-0.017817,-0.008568,-0.009784,-0.011293,-0.009355,0.010979
range_pct,0.232768,0.244924,0.219407,0.231275,0.568686,-0.065306,0.524884,0.021340,1.000000,-0.063018,0.232000,0.021976,0.017836,-0.017221,-0.168180
body_pct,-0.009194,0.003892,0.004947,0.015438,0.055502,0.990235,0.053563,-0.017817,-0.063018,1.000000,-0.006090,0.240397,0.054038,-0.024537,0.004987
